In [12]:
# Units
lb = lbs = 1
gallon = gallons = 1
ounce = ounces = 1

In [13]:
# Constants
hlt_deadspace = 2.25*gallons
mash_deadspace = 0.25*gallons  # what the hell did I mean by this?
hot_side_tubing_deadspace = 0.25*gallons
cold_side_tubing_deadspace = 0.25*gallons
boil_deadspace = 2.25*gallons

In [14]:
# Formulas
mash_thickness = lambda lbs_grains: lbs_grains*(0.375*(gallons/lb))
grain_absorption = lambda lbs_grains: lbs_grains*(0.125*(gallons/lb))
hop_absorption = lambda ounces_hops: ounces_hops*(0.04*(gallons/ounce))
boil_off = lambda boil_hours: (1.5*gallons)*boil_hours

In [15]:
def how_much_water_do_I_need(lbs_grain=0, ounces_hops=0, boil_hours=0, final_expected_volume=5.5):
    dead_liquid = hlt_deadspace + hot_side_tubing_deadspace + cold_side_tubing_deadspace + boil_deadspace
    lost_to_grains = grain_absorption(lbs_grain)
    lost_to_hops = hop_absorption(ounces_hops)
    evaporation = boil_off(boil_hours)
    start_volume = dead_liquid + lost_to_grains + lost_to_hops + evaporation + final_expected_volume
    mash_volume = mash_thickness(lbs_grain) + mash_deadspace + hot_side_tubing_deadspace
    pre_boil_volume = final_expected_volume + evaporation + lost_to_hops + cold_side_tubing_deadspace + boil_deadspace + lost_to_hops
    post_boil_volume = pre_boil_volume - evaporation
    sparge_volume = pre_boil_volume + lost_to_grains + cold_side_tubing_deadspace - mash_volume
    return {
        "start_volume": start_volume,
        "mash_volume": mash_volume,
        "sparge_volume": sparge_volume,
        "pre-boil_volume": pre_boil_volume,
        "post-boil_volume": post_boil_volume,
        "fermenter_volume": final_expected_volume
    }

In [16]:
how_much_water_do_I_need(lbs_grain=18.7, ounces_hops=8, boil_hours=1, final_expected_volume=5.5)

{'fermenter_volume': 5.5,
 'mash_volume': 7.512499999999999,
 'post-boil_volume': 8.64,
 'pre-boil_volume': 10.14,
 'sparge_volume': 5.215000000000002,
 'start_volume': 14.6575}

### Note about working with gravity:
Specific Gravity needs to be converted from the 1.050 format to 50 before doing math with it.
so always multiply and/or divide by 1000
1.050 == 50

In [54]:
# gravity in SG (ie, 1.050)
def what_is_my_expected_post_boil_gravity(pre_boil_volume, pre_boil_gravity, boil_hours):
    post_boil_volume = pre_boil_volume - boil_off(boil_hours)
    converted_gravity = (pre_boil_gravity-1) * 1000
    post_boil_gravity = ((pre_boil_volume*gallons) * converted_gravity) / (post_boil_volume)
    return 1+(post_boil_gravity/1000.0)
                                          

In [55]:
what_is_my_expected_post_boil_gravity(11.8999, 1.039, 1)

1.0446250540870585

In [72]:
# pre-boil gravity = (post-boil_gravity * post_boil_vol) / pre_boil_volume
pre_boil_grav = 1 + ((((1.057 - 1) * 1000 * 8.25) / (8.25+1.5)) / 1000)
pre_boil_grav

1.048230769230769

In [82]:
def what_do_i_need(post_boil_vol=8.25, post_boil_og=1.57, boil_length=1.0, hops=6.0):
    pre_boil_vol = post_boil_vol + boil_off(boil_length) + hop_absorption(hops)
    pre_boil_gravity = 1 + ((((post_boil_og - 1) * 1000 * post_boil_vol) / (pre_boil_vol)) / 1000)
    return "{} gallons of {} gravity wort".format(pre_boil_gravity, pre_boil_vol)

In [83]:
what_do_i_need()

'1.470720720720721 gallons of 9.99 gravity wort'

## How to make a recipe:
- 1. What's the post boil gravity and volume need to be?
- 2. Use that to figure out what the pre-boil gravity and pre-boil volume needs to be
- 3. Use the brewers friend recipe builder to dial in the given boil size with an efficiency of 72%